# 1: Data Processing

## 1.1 Data Gathering

### Introduction

Public bike rental schemes have been launched in many cities around Europe, including successful networks in Irish towns and cities such as Dublin, Cork, and Galway. These provide local active transport links for commuting and leisure. This project explores the publically available data pertaining to the Dublin bike network, makes comparisons to the London network run by Santander, and makes recommendations on network adjustments that could be made to improve the network.

There are several data sources to pull from and process:

Data pertaining to the Dublin bike scheme - number of hires, availability, locations of stations, etc.
- https://data.gov.ie/dataset/dublinbikes-api - Historical data for Dublin bike hire (CSV format)
- https://developer.jcdecaux.com/#/opendata/vls?page=static&contract=dublin - (JSON format)

Similar data pertaining to the London bike scheme in order provide a point of comparison.
- https://data.london.gov.uk/dataset/number-bicycle-hires - The number of bicycle hires in London (XLS format)
- https://data.london.gov.uk/dataset/cycle-hire-availability - The availability of bike hires in London

Population data for the Dublin area. It is possible that the population in the vicinity of bike stations plays a significant role in their usage/availability, so this is gathered to enable features to be derived relating to this.
- https://ie-cso.maps.arcgis.com/apps/webappviewer/index.html?id=0fe164e96d254776866425e2fd3e73af - Population data for Dublin (GIS data)
- https://sdi.eea.europa.eu/catalogue/srv/eng/catalog.search#/metadata/3c362237-daa4-45e2-8c16-aaadfb1a003b - GIS grid data for Ireland (GIS data / parquet)

Weather data for Dublin - it is possible that the weather (temperature/rainfall) has a role in bike usage, so this is gathered in order to explore this link.
- https://www.met.ie/climate/available-data/historical-data - Historical weather data for Dublin (CSV format)

In [ ]:
## unquote to install dependencies from file if not already installed
# pip install -r requirements.txt

In [26]:
import os
import requests
from pathlib import Path
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from tqdm import tqdm

warnings.filterwarnings('ignore')

# Flag to control redownloading - avoids unnecessary web traffic
REDOWNLOAD = False

data_folder = Path('data/')
dublin_bike_folder = Path('data/dublinbikes')

if not data_folder.exists():
    data_folder.mkdir()
    
if not dublin_bike_folder.exists():
    dublin_bike_folder.mkdir()
    (dublin_bike_folder / 'monthly').mkdir()
    (dublin_bike_folder / 'quarterly').mkdir()

In [8]:
# dublinbikes csv data
main_page_url = 'https://data.gov.ie/dataset/dublinbikes-api'
main_page_html = requests.get(main_page_url).content

In [17]:
soup = BeautifulSoup(main_page_html)
download_links = [a['href'] for a in soup.find_all('a', href=True) if a['href'].endswith('.csv') and 'dublinbike' in a['href']]
download_links 

['https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/9496fac5-e4d7-4ae9-a49a-217c7c4e83d9/download/dublinbikes_20180701_20181001.csv',
 'https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/67ea095f-67ad-47f5-b8f7-044743043848/download/dublinbikes_20181001_20190101.csv',
 'https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/538165d7-535e-4e1d-909a-1c1bfae901c5/download/dublinbikes_20190101_20190401.csv',
 'https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/76fdda3d-d8be-441b-92dd-0ee36d9c5316/download/dublinbikes_20190401_20190701.csv',
 'https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/305d39ac-b6a0-4216-a535-0ae2ddf59819/download/dublinbikes_20190701_20191001.csv',
 'https://data.smartdublin.ie/dataset/33ec9fe2-4957-4e9a-ab55-c5e917c7a9ab/resource/5d23332e-4f49-4c41-b6a0-bffb77b33d64/download/dublinbikes_20191001_20200101.csv',
 'ht

In [27]:
# Warning: this downloads several Gbs of bike data
quarterly_links = [x for x in download_links if 'dublinbikes' in x]
monthly_links = [x for x in download_links if 'dublinbikes' not in x]

if REDOWNLOAD:
    for link in tqdm(quarterly_links):
        df = pd.read_csv(link)
        df.to_csv((dublin_bike_folder / 'quarterly' / os.path.basename(link)), index=False)
                  
    for link in tqdm(monthly_links):
        df = pd.read_csv(link)
        df.to_csv((dublin_bike_folder / 'monthly' / os.path.basename(link)), index=False)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27/27 [01:01<00:00,  2.28s/it]
